<a href="https://colab.research.google.com/github/AARABAabdallah/RNAclassification/blob/master/RNA_Embedding_without_padd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model with Embedding layer and Without zero padding
X, y = pd.read_csv(file.csv)

X_train, y_train, X_test, y_test, X_val, y_val = train_test_split(X,y)

X_train[0] = 'AAGAU...C'

X_train = strings_to_numbers(X_train):
  
  'A' --> 1

  'C' --> 2

  'G' --> 3

  'U' --> 4


==> X_train[0] = [1, 1, 3, 4, ..., 2]

![Texte alternatif…](https://scontent.fyhu1-1.fna.fbcdn.net/v/t1.15752-9/87955666_2389668877924663_2448966025484435456_n.jpg?_nc_cat=101&_nc_ohc=jM8zKxlBIP0AX8uxWYP&_nc_ht=scontent.fyhu1-1.fna&oh=2d20e01c9002270b2f38f75f0263760b&oe=5EC4C667)






In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm import tnrange
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pickle

In [0]:
data = pd.read_csv('drive/My Drive/Colab Workspace/RNA_classification/DataSet/All_RNA_Families_IdSeqTotal_IdFamily_Seq.csv')
#Split data
X = np.array(data.sequence)
y = np.array(data.id_family)
family_labels = np.unique(np.array(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
#Données de validations !
X_val1, X_val, y_val1, y_val = train_test_split(X_test, y_test, test_size=0.17)
#len(X_val)

In [0]:
X_train_copy = np.copy(X_train)
X_test_copy = np.copy(X_test)
X_val = np.copy(X_val)

In [0]:
X_train = np.copy(X_train_copy)
X_test = np.copy(X_test_copy)
X_val = np.copy(X_val)

In [36]:
X_train[0]

'UGCCCCUUGGCAAAAAUUAGUCGCUCUGGGUUGGCUUAGUAGGUUAAUUCUCGAAAGGUCGGCAGGCAAAUAUAAG'

In [0]:
def strings_to_numbers(data):
  output = []
  for i in tnrange(len(data)):
    string_to_number_array = []
    for l in list(data[i]):
      if l == 'A':
        string_to_number_array.append(0)
      elif l == 'C':
        string_to_number_array.append(1)
      elif l == 'G':
        string_to_number_array.append(2)
      else:
        string_to_number_array.append(3)
    
    output.append(np.array(string_to_number_array))
  return np.array(output)

In [38]:
X_train = strings_to_numbers(X_train)
X_test = strings_to_numbers(X_test)
X_val = strings_to_numbers(X_val)

In [39]:
X_train[0].shape

(76,)

In [0]:
y_train_copy = np.copy(y_train)
y_test_copy = np.copy(y_test)
y_val_copy = np.copy(y_val)

In [0]:
y_train = np.copy(y_train_copy)
y_test = np.copy(y_test_copy)
y_val = np.copy(y_val_copy)

In [42]:
print(len(np.unique(y_train)))
print(len(np.unique(y_val)))

3016
1175


In [0]:
# OneHot Encoding labels: Just for now !!!
def one_hot_encode_labels(labels):
  output = []
  for i in tnrange(len(labels)):
    output_i = np.zeros(len(family_labels), dtype=int)
    output_i[labels[i]] = 1
    output.append(output_i)
  return np.array(output)

In [44]:
y_train = one_hot_encode_labels(y_train)
y_test = one_hot_encode_labels(y_test)
y_val = one_hot_encode_labels(y_val)

# Model Training

In [45]:
# Model Creation
from keras.models import Sequential
from keras.layers import LSTM, Input, Dense, Activation, Embedding, Bidirectional, Dropout

model = Sequential()
#model.add(Embedding(input_dim=4, output_dim=64, input_length=None))
model.add(Embedding(input_dim=4, output_dim=32, input_length=None))
model.add(Bidirectional(LSTM(units=64, return_sequences=False, input_shape=(None, 32))))
#model.add(LSTM(units=64, return_sequences=False, input_shape=(None, 64)))
#model.add(LSTM(units=64, return_sequences=False, input_shape=(None, 32)))
model.add(Dropout(0.5))
model.add(Dense(len(family_labels),name='out_layer', activation='softmax'))
model.summary(90)

Model: "sequential_6"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
embedding_6 (Embedding)                 (None, None, 32)                    128           
__________________________________________________________________________________________
bidirectional_4 (Bidirectional)         (None, 128)                         49664         
__________________________________________________________________________________________
dropout_6 (Dropout)                     (None, 128)                         0             
__________________________________________________________________________________________
out_layer (Dense)                       (None, 3016)                        389064        
Total params: 438,856
Trainable params: 438,856
Non-trainable params: 0
_______________________________________________________________________________________

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
history = model.fit(train_generator_embedding_layer(), steps_per_epoch=len(X_train_init), epochs=10, verbose=1)

In [0]:
def compute_accuracy(data, labels):
  acc = 0
  for i in tnrange(len(data)):
    if np.argmax(model.predict(data[i].reshape(1, len(data[i]), 4))) == np.argmax(labels[i]):
      acc += 1
  return (acc/len(data))*100

def compute_loss(data, labels):
  loss = 0
  for i in tnrange(len(data)):
    x = data[i].reshape(1, len(data[i]), 4)
    y = labels[i].reshape(1,len(labels[i]))
    loss += model.evaluate(x, y, verbose=0)
  return loss/len(data)


In [0]:
X_train_init = X_train
y_train_init = y_train
data_lentgh = len(X_train_init)
def train_generator_embedding_layer():

    i = 0
    while True:
        if i == data_lentgh: #End of one epoch
          i = 0
        j = data_lentgh - 1 - i
        #x_tr = (X_train_init[j]).reshape(1, len(X_train_init[j]), 4)
        x_tr = X_train_init[j].reshape(1, len(X_train_init[j]))
        y_tr = y_train_init[j].reshape(1,len(y_train_init[j]))
        yield x_tr, y_tr
        i += 1
        if i % 100 == 0:
          model.save_weights("drive/My Drive/Colab Workspace/RNA_classification/models/LSTM_OneHot_Embedding_without_padd.h5")

In [0]:
#Train epochs
def train_epochs(n_epochs=1, already_trained=True):
  
  for i in range(n_epochs):
    
    if already_trained:
      # !!!!!!! Load old model with its history if the model has been trained before !!!!!!! 
      history_file_name = 'drive/My Drive/Colab Workspace/RNA_classification/models/trainHistoryDict_Embedding_without_padd'
      model_file_name = "drive/My Drive/Colab Workspace/RNA_classification/models/LSTM_OneHot_Embedding_without_padd.h5"
      previous_history = pickle.load(open(history_file_name, "rb"))
      model.load_weights(model_file_name)

    
    # model training with train_generator ==> without batch optimization
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    history = model.fit(train_generator(), steps_per_epoch=len(X_train_init), epochs=1, verbose=1)#, validation_data=(X_val, y_val))

    #Compute train acc, val loss, val acc
    history.history['acc'] = [compute_accuracy(X_train_init, y_train_init)]
    history.history['val_acc'] = [compute_accuracy(X_val, y_val)]
    history.history['val_loss'] = [compute_loss(X_val, y_val)]
    
    if already_trained:
      #concatenate history if its been trained before !!!!!!!!!!
      history.history['acc'] = np.append(previous_history['acc'], history.history['acc'])
      history.history['val_acc'] = np.append(previous_history['val_acc'], history.history['val_acc'])
      history.history['loss'] = np.append(previous_history['loss'], history.history['loss'])
      history.history['val_loss'] = np.append(previous_history['val_loss'], history.history['val_loss'])
      
    model.save_weights(model_file_name)
    with open(history_file_name, 'wb') as file_pi:
          pickle.dump(history.history, file_pi)
    print("epoch: ", i+1)
  return history


In [0]:
def plot_history(history):
  # Plot training & validation accuracy values
  plt.plot(history['acc'])
  plt.plot(history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history['loss'])
  plt.plot(history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.show()

In [0]:
history = train_epochs(n_epochs=6,already_trained=True)
plot_history(history.history)

Epoch 1/1
11268/54039 [=====>........................] - ETA: 45:47 - loss: 3.6552

In [0]:
history_file_name = 'drive/My Drive/Colab Workspace/RNA_classification/models/trainHistoryDict_Embedding_without_padd'
model_file_name = "drive/My Drive/Colab Workspace/RNA_classification/models/LSTM_OneHot_Embedding_without_padd.h5"
previous_history = pickle.load(open(history_file_name, "rb"))
previous_history

#plot_history(previous_history)
#len(previous_history['val_loss'])
#print("number of epochs is:",len(history.history['loss']))
#history.history

FileNotFoundError: ignored

In [0]:
#Test accuracy
accuracy = compute_accuracy(X_test, y_test)
print("the model's accuracy is: ", accuracy, "%")


the model's accuracy is:  54.2275023594071 %
